In [117]:
from keras.layers import *
from keras import Model
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras import Sequential
import keras as kr
import keras.losses as losses
 
 
# Setup the network parameters:
intermediate_dim = 300
latent_dim = 4
beta = 5
n_sequences = 200
n_features = 8

boat_csv = pd.read_csv("Data/Boat_nominal_data/Boat_sequences.csv")
boat_csv = boat_csv.drop(columns=["Unnamed: 0"])
# boat_csv = boat_csv[:5739]
scaler = StandardScaler()
normal_data = scaler.fit_transform(boat_csv)
print(normal_data.shape)

boat_curved = pd.read_csv("Data/Boat_anomalous_big.csv")
boat_curved= boat_curved.drop(columns=["Unnamed: 0"])    
scaler = StandardScaler()
anomalous_data = scaler.fit_transform(boat_curved)
print(anomalous_data.shape)


def prepare_sequences(data, batch_size, interval):
    samples = []
    for i in range(0,data.shape[0]- batch_size, interval):
        sample = data[i:i+batch_size]	
        samples.append(sample)

    sequences = np.array(samples)

    # Batch size (Number of samples time steps and number of features
    trainX = np.reshape(sequences, (len(sequences), batch_size, n_features))

    return trainX

trainX_nominal = prepare_sequences(normal_data,656,656) 
print(trainX_nominal.shape)

input_length = trainX_nominal.shape[0]
trainX_anomalous = prepare_sequences(anomalous_data,6600,6600)
print(trainX_anomalous.shape)  


(66256, 8)


(72600, 8)
(100, 656, 8)
(10, 6600, 8)


In [120]:

def create_model(pool_size,up_size, windows, seq_lenght, trainX, epochs):
    x = Input(shape=(seq_lenght, n_features), name="input")
    h = x
    h = Conv1D(filters=300, kernel_size=n_features,
               activation='relu',
               padding='same', name='Conv1')(h)
    h = MaxPooling1D(pool_size=pool_size[0], name='Maxpool1')(h)
    h = Conv1D(filters=200, kernel_size=n_features,
               activation='relu',
               padding='same', name='Conv2')(h)
    h = MaxPooling1D(pool_size=pool_size[1],  name="Maxpool2")(h)
    h = Conv1D(filters=150, kernel_size=n_features,
               activation='relu',
               padding='same',name='Conv3')(h)
    h = MaxPool1D(pool_size=pool_size[2], name="Maxpool3")(h)
    h = Conv1D(filters=100, kernel_size=n_features,
               activation='relu',
               padding="same", name='Conv4')(h)
    h = MaxPool1D(pool_size=pool_size[3], name="Maxpool4")(h)
    h = Conv1D(filters=latent_dim, kernel_size=n_features,
               activation='relu',
               padding="same", name='Conv5')(h)
    h = MaxPool1D(pool_size=pool_size[4], name="Maxpool5")(h)

    h = Flatten()(h)
    
    h = Dense(latent_dim, name='embedding') (h)
    y = h
    
    y = Dense(seq_lenght) (y)
    y = Reshape((windows, np.prod(pool_size)))(y)
    y = Conv1D(filters=n_features, kernel_size=n_features,
               activation='relu',
               padding='same', name='conv-decode1')(y)
    y = UpSampling1D(size=up_size[0], name='upsampling1')(y)
    y = Conv1D(filters=150, kernel_size=n_features,
               activation='relu',
               padding='same', name='conv-decode2')(y)
    y = UpSampling1D(size=up_size[1], name='upsampling2')(y)
    y = Conv1D(filters=200, kernel_size=n_features,
               activation='relu',
               padding='same', name='conv-decode3')(y)
    y = UpSampling1D(size=up_size[2], name='upsampling3')(y)
    y = Conv1D(filters=n_features, kernel_size=n_features,
               padding='same', name='conv-decode4')(y)
    y = UpSampling1D(size=up_size[3], name='upsampling4')(y)        
    
    model = Model(inputs=x, outputs=y, name='AE')
    enc = Model(inputs=x, outputs=h, name='encoder')
    
    model.summary()
    
    model.compile(optimizer="adam", loss="mse", metrics=['accuracy'])
    model.fit(x=trainX, y=trainX, epochs=epochs)
    #model.save("Models/Nominal_conv_compressor.h5")
    
    return model,enc
    
# model, enc = create_model(pool_size=[2,2,2,2],windows=41, 
#                           up_size=[2,2,2,2], seq_lenght=656, 
#                           trainX=trainX_nominal, epochs=15)
model, enc = create_model(pool_size=[2,2,2,5,5], windows=33, 
                          up_size=[5,5,4,2], seq_lenght=6600, 
                          trainX=trainX_anomalous, epochs=50)



# model = Sequential()
# 
# encoder = Sequential()
# encoder.add(Conv1D(filters=200, input_shape=(656,7), padding='same', 
#                    kernel_size=n_features))
# encoder.add(Conv1D(filters=150, padding='same', 
#                    kernel_size=n_features))
# encoder.add(Conv1D(filters=100,padding='same', 
#                    kernel_size=n_features))
# encoder.add(Conv1D(filters=50, padding='same',
#                    kernel_size=n_features))
# encoder.add(Conv1D(filters=latent_dim, padding='same',
#                    kernel_size=n_features))
# #encoder.add(Dense(latent_dim, name="embedding"))
# 
# 
# decoder = Sequential(name="Decoder")
# #decoder.add(Reshape((656,1), name="Reshape"))
# decoder.add(Dense(64, name="Decoder_first"))
# decoder.add(Dense(units=n_features))
# 
# model.add(encoder)
# model.add(decoder)
# 
# model.compile(optimizer="adam", loss="mse", metrics=['accuracy'])
# 
# model.fit(x=trainX_nominal, y=trainX_nominal, epochs=epochs)
#model.save("Models/Nominal_conv_model.h5")


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 6600, 8)           0         
_________________________________________________________________
Conv1 (Conv1D)               (None, 6600, 300)         19500     
_________________________________________________________________
Maxpool1 (MaxPooling1D)      (None, 3300, 300)         0         
_________________________________________________________________
Conv2 (Conv1D)               (None, 3300, 200)         480200    
_________________________________________________________________
Maxpool2 (MaxPooling1D)      (None, 1650, 200)         0         
_________________________________________________________________
Conv3 (Conv1D)               (None, 1650, 150)         240150    
_________________________________________________________________
Maxpool3 (MaxPooling1D)      (None, 825, 150)          0         
__________

Epoch 1/50


10/10 [==============================] - 59s 6s/step - loss: 1.0005 - acc: 0.0956


Epoch 2/50


10/10 [==============================] - 3s 307ms/step - loss: 1.0003 - acc: 0.1547


...

10/10 [==============================] - 3s 329ms/step - loss: 0.4085 - acc: 0.6143


Epoch 49/50


10/10 [==============================] - 3s 312ms/step - loss: 0.4072 - acc: 0.6162


Epoch 50/50


10/10 [==============================] - 3s 318ms/step - loss: 0.4062 - acc: 0.6162


In [121]:

enc_values = enc.predict(trainX_anomalous)
print(enc_values.shape)

plt.plot(enc_values)
plt.show()


(10, 4)


In [122]:
from sklearn.decomposition import PCA

pca=True

if pca:
    scaler = StandardScaler()
    enc_input = scaler.fit_transform(enc_values)
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(enc_input)
    x_val = []
    y_val=[]
    for i in range(principalComponents.shape[0]):
        x_val.append(principalComponents[i][0])
        y_val.append(principalComponents[i][1])
    
    
    plt.scatter(x=x_val,y=y_val, alpha=0.5)
    plt.show()


In [4]:
nom_model = kr.models.load_model("Models/Nominal_conv_model.h5")
anom_model = kr.models.load_model("Models/Anom_conv_model.h5") 

print("Done")

Done


In [5]:

from sklearn.manifold import TSNE

def tsne(data, title):
    tsne = TSNE(n_components=2, random_state=0)
    
    tsne_obj= tsne.fit_transform(data)
    tsne_df = pd.DataFrame({'X':tsne_obj[:,0],
                            'Y':tsne_obj[:,1],
                            })
    
    plt.scatter(x=tsne_df["X"],y=tsne_df["Y"], alpha=0.5)
    plt.title(title)
    plt.show()
    
    return tsne_df


In [16]:
weight_modifiers = np.ones((latent_dim,))

nom_enc =(anom_model.get_layer(index=0).predict(trainX_anomalous))
print(nom_enc.shape)

tsne_enc_df = tsne(nom_enc[0], "Encoded Nominal")
print(len(tsne_enc_df))
nom_dec = nom_model.get_layer(index=1).predict(nom_enc)
tsne_dec_df = tsne(nom_dec[0], "Decoded Nominal")
print(len(tsne_dec_df))


(10, 6620, 10)


6620


6620


In [17]:
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering

plt.figure(figsize=(10, 7))
plt.title("Customer Dendograms")
dend = shc.dendrogram(shc.linkage(tsne_enc_df, method='ward'))


In [23]:

n_clusters = 3
cluster = AgglomerativeClustering(n_clusters=n_clusters, 
                                  affinity='euclidean',
                                  linkage='ward')
cl = cluster.fit_predict(tsne_enc_df)
print(len(cl))
print(cl)

plt.plot(cl)
plt.show()
# weight_modifiers[0] = 1
# nom_enc_modified = nom_enc
# for i, elem in enumerate(nom_enc_modified):
#     nom_enc_modified[i] = elem * weight_modifiers    


# 
# tsne_enc_df = tsne(np.average(nom_enc, axis=0), "Encoded Nominal")
# # 
# # 
# nom_dec = decoder.predict(nom_enc)
# tsne_dec_df = tsne(np.average(nom_dec, axis=0), "Decoded Nominal")


6620
[0 0 0 ... 1 1 1]


In [24]:
df = pd.DataFrame(anomalous_data, columns=boat_curved.columns)

for i in range(n_clusters):
    cluster = np.squeeze(np.argwhere(cl==i))    
    plt.scatter(x=df['lon'][cluster],y=df["lat"][cluster],s=5)
plt.show()



In [32]:
df_nominal = pd.DataFrame(normal_data, columns=boat_csv.columns)
df = pd.DataFrame(nom_enc[0])

plt.figure(1)
axis_list = []

for i in range(df.shape[1]):
    plt.plot(df[i])

plt.show()    

In [39]:
from sklearn.cluster import KMeans
cluster_comp = [0,1,2,3]# print(tsne_enc_df)
# print(tsne_dec_df)

kmeans_normal = KMeans(n_clusters=2, random_state=0).fit(normal_data)
clusters_normal = kmeans_normal.predict(np.average(trainX_nominal,axis=0))

kmeans_enc = KMeans(n_clusters=2, random_state=0).fit(tsne_enc_df)
clusters_enc = kmeans_enc.predict(tsne_enc_df)
plt.plot(clusters_enc)
plt.show()

kmeans_dec = KMeans(n_clusters=2, random_state=0).fit(tsne_dec_df)
clusters_dec = kmeans_dec.predict(tsne_dec_df)

In [40]:
def transform_to_mask(cl_label, clust_obj):
    mask = []
    part = []
    for i, elem in enumerate(clust_obj):
        if elem == cl_label:
            part.append(i)
        else:
            if part:
                mask.append(part)
                part = []
            else:
                pass
    return mask


def get_mask_list(clust_obj):
    mask_list = []
    for i in cluster_comp:
        mask_list.append(transform_to_mask(i, clust_obj))
    return mask_list


masks_normal = np.array(get_mask_list(clusters_normal))

masks_enc = np.array(get_mask_list(clusters_enc))

masks_dec = np.array(get_mask_list(clusters_dec))

masks = (masks_normal, masks_enc,masks_dec)
print(masks_normal.shape, masks_enc.shape, masks_enc.shape)


(4,) (4,) (4,)


In [41]:
def plot_cl(cl, color):
    plt.plot(cl["lon"], cl["lat"], color=color)
        
    
def plot_clusters_on_map():
    titles = ['Nominal Normal','Nominal Encoded','Nominal Decoded']
    color_list = ['blue','green','red','black']
    map = anomalous_data[:6620]
    print(map.shape)
    for k,mask in enumerate(masks):
        for i, elem in enumerate(mask):
            for j in elem:
                cl = pd.DataFrame(map[j], columns=boat_curved.columns)
                plot_cl(cl, color_list[i]) 
        plt.title(titles[k])
        plt.show()
      
               
plot_clusters_on_map()


(6620, 7)
